In [12]:
%load_ext autoreload
%autoreload 2
from notebook import *
# if get something about NUMEXPR_MAX_THREADS being set incorrectly, don't worry.  It's not a problem.

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Branch and branch predictions

## Why do we have "branches" in code?

Consider the following code snippet, how does the compiler translate to instructions?

In [3]:
render_code("loop.c", show="loop0")

// loop.c:3-13 (11 lines)
int loop0(int *data, int size, int threshold, int reps)
{
    int i, j, sum = 0;
    for (j = 0; j < reps; ++j) {
        for (unsigned i = 0; i < size; ++i) {
            if (data[i] >= threshold)
                sum ++;
        }
    }
    return sum;
}

In [5]:
! gcc -S -O0 loop.c
render_code("loop.s", show=["loop0","LFE23"])

// loop.s:3-160 (158 lines)
	.globl	loop0
	.type	loop0, @function
loop0:
.LFB0:
	.cfi_startproc
	endbr64
	pushq	%rbp
	.cfi_def_cfa_offset 16
	.cfi_offset 6, -16
	movq	%rsp, %rbp
	.cfi_def_cfa_register 6
	movq	%rdi, -24(%rbp)
	movl	%esi, -28(%rbp)
	movl	%edx, -32(%rbp)
	movl	%ecx, -36(%rbp)
	movl	$0, -8(%rbp)
	movl	$0, -12(%rbp)
	jmp	.L2
.L6:
	movl	$0, -4(%rbp)
	jmp	.L3
.L5:
	movl	-4(%rbp), %eax
	leaq	0(,%rax,4), %rdx
	movq	-24(%rbp), %rax
	addq	%rdx, %rax
	movl	(%rax), %eax
	cmpl	%eax, -32(%rbp)
	jg	.L4
	addl	$1, -8(%rbp)
.L4:
	addl	$1, -4(%rbp)
.L3:
	movl	-28(%rbp), %eax
	cmpl	%eax, -4(%rbp)
	jb	.L5
	addl	$1, -12(%rbp)
.L2:
	movl	-12(%rbp), %eax
	cmpl	-36(%rbp), %eax
	jl	.L6
	movl	-8(%rbp), %eax
	popq	%rbp
	.cfi_def_cfa 7, 8
	ret
	.cfi_endproc
.LFE0:
	.size	loop0, .-loop0
	.globl	loop1
	.type	loop1, @function
loop1:
.LFB1:
	.cfi_startproc
	endbr64
	pushq	%rbp
	.cfi_def_cfa_offset 16
	.cfi_offset 6, -16
	movq	%rsp, %rbp
	.cfi_def_cfa_register 6
	movq	%rdi, -24(%rbp)
	movl	%esi, -28(%rbp)
	movl	$0, -4(%rbp)
	movl	$0, -8(%rbp)
	jmp	.L9
.L10:
	movl	-8(%rbp), %eax
	cltq
	leaq	0(,%rax,4), %rdx
	movq	-24(%rbp), %rax
	addq	%rdx, %rax
	movl	(%rax), %eax
	addl	%eax, -4(%rbp)
	addl	$1, -8(%rbp)
.L9:
	cmpl	$9, -8(%rbp)
	jle	.L10
	movl	-4(%rbp), %eax
	popq	%rbp
	.cfi_def_cfa 7, 8
	ret
	.cfi_endproc
.LFE1:
	.size	loop1, .-loop1
	.globl	loop2
	.type	loop2, @function
loop2:
.LFB2:
	.cfi_startproc
	endbr64
	pushq	%rbp
	.cfi_def_cfa_offset 16
	.cfi_offset 6, -16
	movq	%rsp, %rbp
	.cfi_def_cfa_register 6
	movq	%rdi, -24(%rbp)
	movl	%esi, -28(%rbp)
	movl	$0, -4(%rbp)
.L14:
	movl	-4(%rbp), %eax
	andl	$1, %eax
	testl	%eax, %eax
	je	.L13
	movl	-4(%rbp), %eax
	cltq
	leaq	0(,%rax,4), %rdx
	movq	-24(%rbp), %rax
	addq	%rdx, %rax
	movl	(%rax), %edx
	movl	-4(%rbp), %eax
	cltq
	leaq	0(,%rax,4), %rcx
	movq	-24(%rbp), %rax
	addq	%rcx, %rax
	addl	%edx, %edx
	movl	%edx, (%rax)
.L13:
	movl	-4(%rbp), %eax
	cltq
	leaq	0(,%rax,4), %rdx
	movq	-24(%rbp), %rax
	addq	%rdx, %rax
	movl	(%rax), %ecx
	movl	-4(%rbp), %eax
	cltq
	leaq	0(,%rax,4), %rdx
	movq	-24(%rbp), %rax
	addq	%rdx, %rax
	movl	-4(%rbp), %edx
	addl	%ecx, %edx
	movl	%edx, (%rax)
	addl	$1, -4(%rbp)
	cmpl	$99, -4(%rbp)
	jle	.L14
	movl	$0, %eax
	popq	%rbp
	.cfi_def_cfa 7, 8
	ret
	.cfi_endproc
.LFE2:
	.size	loop2, .-loop2
	.ident	"GCC: (Ubuntu 13.2.0-23ubuntu4) 13.2.0"
	.section	.note.GNU-stack,"",@progbits
	.section	.note.gnu.property,"a"
	.align 8
	.long	1f - 0f
	.long	4f - 1f
	.long	5
0:
	.string	"GNU"
1:
	.align 8
	.long	0xc0000002
	.long	3f - 2f
2:
	.long	0x3
3:
	.align 8
4:

In [6]:
! gcc -S -O2 loop.c
render_code("loop.s", show=["loop1","LFE24"])

// loop.s:52-69 (18 lines)
	.globl	loop1
	.type	loop1, @function
loop1:
.LFB24:
	.cfi_startproc
	endbr64
	leaq	40(%rdi), %rdx
	xorl	%eax, %eax
	.p2align 4,,10
	.p2align 3
.L16:
	addl	(%rdi), %eax
	addq	$4, %rdi
	cmpq	%rdx, %rdi
	jne	.L16
	ret
	.cfi_endproc
.LFE24:

In [5]:
! gcc -S -O2 loop.c
render_code("loop.s", show=["loop2","LFE25"])

// loop.s:66-95 (30 lines)
	.globl	loop2
	.type	loop2, @function
loop2:
.LFB25:
	.cfi_startproc
	endbr64
	movl	(%rdi), %eax
	leaq	4(%rdi), %rcx
	xorl	%edx, %edx
	jmp	.L16
	.p2align 4,,10
	.p2align 3
.L18:
	movl	(%rcx), %eax
	testb	$1, %dl
	je	.L17
	addl	%eax, %eax
	movl	%eax, (%rcx)
.L17:
	addq	$4, %rcx
.L16:
	addl	%edx, %eax
	addl	$1, %edx
	movl	%eax, -4(%rcx)
	cmpl	$100, %edx
	jne	.L18
	xorl	%eax, %eax
	ret
	.cfi_endproc
.LFE25:

## Sorting and branch miss rates

Do you remember this?

In [13]:
compare([do_render_code("arraySort.cpp",show=["//START","//END"]),do_render_code("calculate_sum.c", show="calculate_sum")])

In [14]:
! lscpu | grep 'Model name'

Model name:                           13th Gen Intel(R) Core(TM) i7-13700


In [15]:
! make clean; make EXTRA_OPTS=-DCOUNT_SORTING; sleep 2
! echo "size,iterations,sorted,IC,Cycles,CPI,CT,ET,L1_dcache_miss_rate,L1_dcache_misses,L1_dcache_accesses,branches,branch_misses" > stats.csv
! echo -n "131072,1000,0," >> stats.csv
! ./arraySort 131072 1000 0
! echo -n "131072,1000,1," >> stats.csv
! ./arraySort 131072 1000 1

rm -f madd arraySort *.o
gcc -g -DHAVE_LINUX_PERF_EVENT_H -O0 -DCOUNT_SORTING -I/nfshome/htseng/courses/CS203/demo/branch  -o calculate_sum.o -c calculate_sum.c
gcc -g -DHAVE_LINUX_PERF_EVENT_H -O3 -DCOUNT_SORTING -I/nfshome/htseng/courses/CS203/demo/branch  -o perfstats.o -c perfstats.c
perfstats.c: In function ‘change_cpufrequnecy’:
perfstats.c:115:11: warning: implicit declaration of function ‘sched_getcpu’; did you mean ‘SYS_getcpu’? []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wimplicit-function-declaration-Wimplicit-function-declaration]8;;]
  115 |     cpu = sched_getcpu();
      |           ^~~~~~~~~~~~
      |           SYS_getcpu
g++ -O3 -DHAVE_LINUX_PERF_EVENT_H -DCOUNT_SORTING arraySort.cpp perfstats.o calculate_sum.o -o arraySort
arraySort.cpp: In function ‘int main(int, char**)’:
arraySort.cpp:53:22: warning: zero-length gnu_printf format string []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wformat-zero-length-Wformat-zero-

In [16]:
display_df_mono(render_csv("stats.csv"))

,index,size,iterations,sorted,IC,Cycles,CPI,CT,ET,L1_dcache_miss_rate,L1_dcache_misses,L1_dcache_accesses,branches,branch_misses
0,0,131072,1000,0,1500089369,1560707085,1.040409,0.197792,0.308695,0.007012,8216848,1171809488,261041614,35149848
1,1,131072,1000,1,1522559118,375015986,0.246306,0.196840,0.073818,0.006986,8260554,1182512878,266006670,957531


In [70]:
miss_prediction_rate_without_sorting = 35372534/262712331
print(miss_prediction_rate_without_sorting)
cycles_with_sorting = 1560707085

0.134643599961054


What did we learn?

The CPI is ??? smaller with data sorted (including sorting itself)

The ET is ??? faster with data sorted (including sorting itself)

What's the cost of branch misses?

Let's exclude the sorting part and do it again.

In [17]:
! make clean; make
! sleep 2
! echo "size,iterations,sorted,IC,Cycles,CPI,CT,ET,L1_dcache_miss_rate,L1_dcache_misses,L1_dcache_accesses,branches,branch_misses" > stats.csv
! echo -n "262144,1000,0," >> stats.csv
! ./arraySort 262144 1000 0
! echo -n "262144,1000,1," >> stats.csv
! ./arraySort 262144 1000 1
display_df_mono(render_csv("stats.csv"))

rm -f madd arraySort *.o
gcc -g -DHAVE_LINUX_PERF_EVENT_H -O0  -I/nfshome/htseng/courses/CS203/demo/branch  -o calculate_sum.o -c calculate_sum.c
gcc -g -DHAVE_LINUX_PERF_EVENT_H -O3  -I/nfshome/htseng/courses/CS203/demo/branch  -o perfstats.o -c perfstats.c
perfstats.c: In function ‘change_cpufrequnecy’:
perfstats.c:115:11: warning: implicit declaration of function ‘sched_getcpu’; did you mean ‘SYS_getcpu’? []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wimplicit-function-declaration-Wimplicit-function-declaration]8;;]
  115 |     cpu = sched_getcpu();
      |           ^~~~~~~~~~~~
      |           SYS_getcpu
g++ -O3 -DHAVE_LINUX_PERF_EVENT_H  arraySort.cpp perfstats.o calculate_sum.o -o arraySort
arraySort.cpp: In function ‘int main(int, char**)’:
arraySort.cpp:53:22: warning: zero-length gnu_printf format string []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wformat-zero-length-Wformat-zero-length]8;;]
   53 |     sprintf(preamble,""

,index,size,iterations,sorted,IC,Cycles,CPI,CT,ET,L1_dcache_miss_rate,L1_dcache_misses,L1_dcache_accesses,branches,branch_misses
0,0,262144,1000,0,3015678570,3013721621,0.999351,0.196537,0.592308,0.004270,10048477,2353311459,525147647,67979099
1,1,262144,1000,1,3011998253,707433688,0.234872,0.196626,0.139100,0.006609,15543808,2351953168,524510162,4686


In [18]:
Diff_Misses = 67979099-4686
Diff_Cycles = 3013721621-707433688
print(Diff_Cycles/Diff_Misses)

33.92876571070941


Let's try a different processor!

In [19]:
! make clean; make
! sleep 2
! echo "size,iterations,sorted,IC,Cycles,CPI,CT,ET,L1_dcache_miss_rate,L1_dcache_misses,L1_dcache_accesses,branches,branch_misses" > stats.csv
! ssh htseng@blissey 'make -C /nfshome/htseng/courses/CSE142/demo/branch/ clean all ; lscpu|grep "Model name"'
! echo -n "262144,1000,0," >> stats.csv
! ssh htseng@blissey "cd /nfshome/htseng/courses/CSE142/demo/branch; ./arraySort 262144 1000 0"
! echo -n "262144,1000,1," >> stats.csv
! ssh htseng@blissey "cd /nfshome/htseng/courses/CSE142/demo/branch; ./arraySort 262144 1000 1"
display_df_mono(render_csv("stats.csv"))

rm -f madd arraySort *.o
gcc -g -DHAVE_LINUX_PERF_EVENT_H -O0  -I/nfshome/htseng/courses/CS203/demo/branch  -o calculate_sum.o -c calculate_sum.c
gcc -g -DHAVE_LINUX_PERF_EVENT_H -O3  -I/nfshome/htseng/courses/CS203/demo/branch  -o perfstats.o -c perfstats.c
perfstats.c: In function ‘change_cpufrequnecy’:
perfstats.c:115:11: warning: implicit declaration of function ‘sched_getcpu’; did you mean ‘SYS_getcpu’? []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wimplicit-function-declaration-Wimplicit-function-declaration]8;;]
  115 |     cpu = sched_getcpu();
      |           ^~~~~~~~~~~~
      |           SYS_getcpu
g++ -O3 -DHAVE_LINUX_PERF_EVENT_H  arraySort.cpp perfstats.o calculate_sum.o -o arraySort
arraySort.cpp: In function ‘int main(int, char**)’:
arraySort.cpp:53:22: warning: zero-length gnu_printf format string []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wformat-zero-length-Wformat-zero-length]8;;]
   53 |     sprintf(preamble,""

,index,size,iterations,sorted,IC,Cycles,CPI,CT,ET,L1_dcache_miss_rate,L1_dcache_misses,L1_dcache_accesses,branches,branch_misses
0,0,262144,1000,0,262144,1000,1,nan,nan,nan,nan,nan,nan,nan


In [33]:
Diff_Misses = 49159652-57646
Diff_Cycles = 1941551579-789330059
print(Diff_Cycles/Diff_Misses)

23.46587469359195


In [35]:
new_CPI = 0.9*1+0.1*24
print(new_CPI)

3.3000000000000003
